In [0]:
!pip install bayesian-optimization

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
pretrain = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/Pretrain.csv')
pretest = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/Pretest.csv')

attack_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/attack_train.csv')
attack_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/attack_test.csv')
tpq_train_0 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/tpq_train_0.csv')
tpq_train_1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/tpq_train_1.csv')
tpq_test_0 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/tpq_test_0.csv')
tpq_test_1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/tpq_test_1.csv') 

In [0]:
attack_train.drop('Unnamed: 0',axis=1,inplace=True)
attack_test.drop('Unnamed: 0',axis=1,inplace=True)
pretrain.drop('Unnamed: 0',axis=1,inplace=True)
pretest.drop('Unnamed: 0',axis=1,inplace=True)
tpq_train_0.drop('Unnamed: 0',axis=1,inplace=True) 
tpq_train_1.drop('Unnamed: 0',axis=1,inplace=True) 
tpq_test_0.drop('Unnamed: 0',axis=1,inplace=True) 
tpq_test_1.drop('Unnamed: 0',axis=1,inplace=True) 

In [0]:
train_data=attack_train.merge(pretrain,on='game_id').merge(tpq_train_0,on='game_id').merge(tpq_train_1,on='game_id')
test_data=attack_test.merge(pretest,on='game_id').merge(tpq_test_0,on='game_id').merge(tpq_test_1,on='game_id')

In [0]:
X = train_data.drop(['winner','game_id'],axis=1)
y= train_data['winner']
X_test = test_data.drop('game_id',axis=1)

# LGBM

In [0]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='model'):
    score = 0
    kf = KFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
       #print(train_index)
       x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
       x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
    model = lgb.LGBMClassifier(
          num_leaves = int(num_leaves), 
          learning_rate = learning_rate, 
          n_estimators = int(n_estimators), 
          subsample = np.clip(subsample, 0, 1), 
          colsample_bytree = np.clip(colsample_bytree, 0, 1), 
          reg_alpha = reg_alpha, 
          reg_lambda = reg_lambda,
      )
    #print(len(x_train))
    #print(len(y_train))
    model.fit(x_train, y_train)
    models.append(model)
        
    pred = model.predict_proba(x_valid)[:, 1]
    true = y_valid
    score += roc_auc_score(true, pred)/n_splits
    
    # results, bad_features = permutation_importance(model=model,
    #                                            X_val=X_valid_1,
    #                                            y_val=y_valid_1,
    #                                            metric=metrics.auc,
    #                                            verbose=False)
    # print(bad_features)
    # bad.append(bad_features)


    if output == 'score':
        return score
    if output == 'model':
        return models

In [0]:
func_fixed = partial(lgb_cv, x_data=X, y_data=y, n_splits=5, output='score') 

lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (16, 1024),
        'learning_rate': (0.0001, 0.1),
        'n_estimators': (16, 2048),#1024),
        'subsample': (0, 1),
        'colsample_bytree': (0, 1),
        'reg_alpha': (0, 10), 
        'reg_lambda': (0, 50),
    }, 
    random_state=4321
)
lgbBO.maximize(init_points=5, n_iter=30) #30

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.1432   |  0.0708   |  0.08152  |  1.576e+0 |  304.6    |  1.931    |  48.95    |  0.4062   |
|  2        |  0.1431   |  0.7578   |  0.009006 |  645.7    |  639.9    |  4.599    |  10.92    |  0.6635   |
|  3        |  0.1426   |  0.6787   |  0.09504  |  587.5    |  640.8    |  3.833    |  20.02    |  0.9427   |
|  4        |  0.1431   |  0.9299   |  0.09484  |  779.0    |  361.0    |  6.648    |  2.116    |  0.2322   |
|  5        |  0.1442   |  0.4301   |  0.007886 |  1.574e+0 |  876.7    |  1.504    |  5.057    |  0.2709   |
|  6        |  0.1443   |  0.6384   |  0.00669  |  2.045e+0 |  1.006e+0 |  1.814    |  44.68    |  0.8205   |
|  7        |  0.1445   |  0.1727   |  0.02629  |  2.045e+0 |  25.59    |  9.732    |  7.839    |  0.7189   |
|  8      

In [0]:
params = lgbBO.max['params']
models = lgb_cv(
    params['num_leaves'],   
    params['learning_rate'], 
    params['n_estimators'], 
    params['subsample'], 
    params['colsample_bytree'], 
    params['reg_alpha'], 
    params['reg_lambda'], 
    x_data=X, y_data=y, n_splits=5, output='model')

In [0]:
sample_submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/sample_submission.csv')
sample_submission.head()

,game_id,winner
0,38872,0
1,38873,0
2,38874,0
3,38875,0
4,38876,0


In [0]:
preds = []
for model in models:
    pred = model.predict(X_test)#predict_proba(test_best)[:, 1]
    preds.append(pred)
pred = np.mean(preds, axis=0)

sample_submission['winner'] = sample_submission['winner'] + pred
sample_submission.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/submission(3).csv')
sample_submission.head()

,game_id,winner
0,38872,0.0
1,38873,1.0
2,38874,0.0
3,38875,0.0
4,38876,1.0
